### Handling Exceptions

We can trap and handle exceptions in sections of code that we include in a `try` statement, catching various exceptions using the `except` clauses:

In [1]:
try:
    1 / 0
except ZeroDivisionError as ex:
    print(f'Exception occured: {type(ex)}, {ex}')
print('code continues running here...')

Exception occured: <class 'ZeroDivisionError'>, division by zero
code continues running here...


Since we "handled" the exception, the exception is silenced, Jupyter is unaware of it, and the code in the cell continues running normally.

Exceptions have a hierarchy, with `Exception` being a broad exception that encompasses many other exceptions.

We don't typically catch exceptions at that broad of a level, simply because we probably don't know how to handle it.

Let's look at example where that may happen.

Suppose we are looking to take an element from an iterable, one by one, until the iterable is empty.

One way to do it is this:

In [2]:
l = [1, 2, 3, 4, 5]

while len(l) > 0:
    print(l.pop())
    
print('all done.')

5
4
3
2
1
all done.


But another way we could do this is to use an exception handler to trap an `IndexError` exception.

Let's try this code withouh an exception handler first:

In [3]:
l = [1, 2, 3, 4, 5]

while True:
    print(l.pop())

print('all done.')

5
4
3
2
1


IndexError: pop from empty list

As you can see an exception (`IndexError`) was raised, and our code was terminated (the `print` statement never executed).

Now, let's add an exception handler for an `IndexError`:

In [4]:
l = [1, 2, 3, 4, 5]

try:
    while True:
        print(l.pop())
except IndexError:
    # index error means our list is empty - that was expected to happen
    print('all done')

5
4
3
2
1
all done


As you can see, the `print('all done')` statement was executed.

This was a case where we were expecting an exception, specifically an `IndexError` exception to happen - and we knew how to handle it - we printed something and let our code resume normally.

Now, we could have trapped it using a broader exception, such as `LookupError`, or even `Exception`:

In [5]:
l = [1, 2, 3, 4, 5]

try:
    while True:
        print(l.pop())
except Exception:
    # hmm, was this an index error, or possibly something else...
    # how do we handle this?
    print('something unexpected may have happened')

5
4
3
2
1
something unexpected may have happened


Now, we can't just assume that the Exception will always be an `IndexError` - if it is, great, that's expected and we know what to do. But what if it was something else? So, something like this is a **bad** idea:

In [6]:
l = [1, 2, 3, 4, 5]

try:
    while True:
        print(l.pop())
except Exception:
    print('all done')

5
4
3
2
1
all done


It actually worked correctly in this case, but what about this:

In [7]:
l = (1, 2, 3, 4, 5)

try:
    while True:
        print(l.pop())
except Exception:
    print('all done')

all done


Hmm - as you can see, we printed `all done`, as if the code had worked correctly, when in fact something else happened - and we failed to recognize it.

Can you figure out what actually went wrong here?

(Hint: what's the type of `l` in this example?)

Instead we should make our exception handlers as specific as possible:

In [8]:
l = (1, 2, 3, 4, 5)

try:
    while True:
        print(l.pop())
except IndexError:
    print('all done')

AttributeError: 'tuple' object has no attribute 'pop'

Now, our code will work properly when it encounters an `IndexError`, and will fail, as it should, when it encounters something else - much easier to debug our code!

We can actually trap multiple exception types in our handlers.

First let's see what happens if we try to add a string to an integer say:

In [9]:
10 + 'abc'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

We get a `TypeError`.

Now suppose we are given a list (from somewhere) that contains data we want to average.

We may not have control over the data in that list, but we may not want out program to fail if the list contains non-numeric data - we may want to set the average to zero and move on.

Additionally, we may get an empty list - in which case we're going to run into an issue trying to calculate the average (a division by zero exception possibly):

The naive way to implement this, without exception handling might be:

In [10]:
data = [10, 20, 10, 10, 5, 10]

sum_data = 0
count_data = 0

for element in data:
    sum_data = sum_data + element
    count_data = count_data + 1
    
average = sum_data / count_data
print(f'average = {average}')

average = 10.833333333333334


That worked just fine, but what happens if our list is empty?

In [11]:
data = []

sum_data = 0
count_data = 0

for element in data:
    sum_data = sum_data + element
    count_data = count_data + 1
    
average = sum_data / count_data
print(f'average = {average}')

ZeroDivisionError: division by zero

We get a `ZeroDivisionError` - we could fix this by using a LBYL approach:

In [12]:
data = []

if len(data) == 0:
    average = 0
else:
    sum_data = 0
    count_data = 0

    for element in data:
        sum_data = sum_data + element
        count_data = count_data + 1

    average = sum_data / count_data
print(f'average = {average}')

average = 0


So now that works fine - but what about if one of the elements is not a number?

We would have to test for that - but that's actually complicated, and in reality, we are just looking for the ability to sum up the elements and divide them by their count - irrespective of whether they are floats, integers, complex numbers, etc.

So, instead of testing things at every loop, or maybe testing things before we even run the averaging code, we use EAFP and just do it, using exception handlers to handle two exceptions we expect may happen - `ZeroDivisionError` and `TypeError`, and importantly, that we know how to handle.

We could do it this way:

In [13]:
data = []

sum_data = 0
count_data = 0

try:
    for element in data:
        sum_data = sum_data + element
        count_data = count_data + 1

    average = sum_data / count_data
except ZeroDivisionError:
    average = 0
except TypeError:
    # hmm. what to do here?
    average = 0
print(f'average = {average}')

average = 0


In [14]:
data = [10, 20, 'a']

sum_data = 0
count_data = 0

try:
    for element in data:
        sum_data = sum_data + element
        count_data = count_data + 1

    average = sum_data / count_data
except ZeroDivisionError:
    average = 0
except TypeError:
    # hmm. what to do here?
    average = 0
print(f'average = {average}')

average = 0


So that worked, but maybe we'd prefer to just "skip" elements that can't be added.

So, we could do this instead:

In [15]:
data = [10, 20, 'a']

sum_data = 0
count_data = 0

try:
    for element in data:
        try:
            sum_data = sum_data + element
            count_data = count_data + 1
        except TypeError:
            # skip element - so do nothing
            pass

    average = sum_data / count_data
except ZeroDivisionError:
    average = 0

print(f'average = {average}')

average = 15.0


In [16]:
data = []

sum_data = 0
count_data = 0

try:
    for element in data:
        try:
            sum_data = sum_data + element
            count_data = count_data + 1
        except TypeError:
            # skip element - so do nothing
            pass

    average = sum_data / count_data
except ZeroDivisionError:
    average = 0

print(f'average = {average}')

average = 0


Generally we don't try to handle broad exceptions, because we can't really recover from them>

But there are cases, where we are not interested in recovering from an error so much as recording the fact that the exception happened, and then either silencing the exception, or re-assign an exception (the same one or a new one).

A typical example of this might be an exception logger where we want to catch a broad set of exceptions, log the error, and then re-start the error flow:

In [17]:
try:
    1 / 0
except Exception as ex:
    print(f'logging error: {ex}')
    
print('program still running...')

logging error: division by zero
program still running...


With this code, we intercepted the exception and handled it - this means the exception flow was interrupted, so Python happily continues executing our code after the handler has completed.

But that's not what we want - we want the exception flow to continue as if we had not interrupted it - so we need to make sure we re-raise the exception in our handler:

In [18]:
try:
    1 / 0
except Exception as ex:
    print(f'logging error: {ex}')
    raise  # or raise ex - same thing
    
print('program still running...')

logging error: division by zero


ZeroDivisionError: division by zero

As you can see, we logged the exception, then re-raised it - and Jupyter handled it as it does normally (prints the exception, with a trace, and stops executing the remaining statements in the cell)

We'll come back to this technique, and logging, when we study decorators later in this course.

One final thing, is the `finally` block - it is guaranteed to run no matter what happens in `try` or `except` blocks - very handy for cleaning up resources - like closing a file that's been opened, closing a database connection, etc.

We'll see this more when we study context managers, but for now let's just see how `finally` works:

In [19]:
try:
    raise ValueError('custom message')
except ValueError as ex:
    print(f'handled a ValueError: {ex}')
finally:
    print('this always executes')
    
print('all done')

handled a ValueError: custom message
this always executes
all done


If we do not handle the exception:

In [20]:
try:
    raise TypeError('custom message')
except ValueError as ex:
    print(f'handled a ValueError: {ex}')
finally:
    print('this always executes')
    
print('all done')

this always executes


TypeError: custom message

As you can see, the exception was unhandled, but the `finally` block still ran.

Same thing happens if we catch the exception and raise it (or a new one):

In [21]:
try:
    raise ValueError('some value error')
except ValueError as ex:
    print(f'handled a ValueError: {ex}')
    raise TypeError('changing to a type error')
finally:
    print('this always executes')
    
print('all done')

handled a ValueError: some value error
this always executes


TypeError: changing to a type error

Again, the `finally` block ran.

Same thing if no exception happens:

In [22]:
try:
    print('Nothing to see here...')
except ValueError as ex:
    print(f'handled a ValueError: {ex}')
finally:
    print('this always executes')
    
print('all done')

Nothing to see here...
this always executes
all done
